In [ ]:
import torch
import torch.linalg as tla
import numpy as np
import matplotlib.pyplot as plt
import numml.sparse as sp

In [ ]:
# Create our favorite poisson operator

N = 9
A = sp.eye(N)*2. - sp.eye(N, k=1) - sp.eye(N, k=-1)
b = torch.zeros(N)
b[N//2] = 1.
print(A.to_dense())

In [ ]:
# Create a tentative aggregation operator

Agg = torch.Tensor([
    [1, 1, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 1, 1]
]).T
Agg = sp.SparseCSRTensor(Agg)

In [ ]:
# Optimize a smoother for the agg op

S = A.copy()
S_orig = S.copy()
S.requires_grad = True

In [ ]:
# multigrid cycle and jacobi smoother

def jacobi(A, x, b, omega=0.6):
    Dinv = sp.diag(1./A.diagonal())
    return omega * (Dinv @ b) + (sp.eye(A.shape[0]) - omega * (Dinv@A)) @ x

def mgrid(P, A, x, b):
    x = jacobi(A, x, b)
    r_H = P.T @ (b - A @ x)
    A_H = P.T @ A @ P
    x = x + P @ sp.spsolve(A_H, r_H)
    x = jacobi(A, x, b)
    return x

def mgrid_solver(P, A, x, b, iterations=100):
    res_history = torch.zeros(iterations)
    res_history[0] = tla.norm(b-A@x)/tla.norm(b)
    
    for i in range(1, iterations):
        x = mgrid(P, A, x, b)
        res_history[i] = tla.norm(b-A@x)/tla.norm(b)
    return res_history

In [ ]:
# Optimize over the entries of S to create an optimal smoother
# we'll use one multigrid iteration for the loss

optimizer = torch.optim.Adam([S.data], lr=0.1)
batch = 20
lh = []

x_test = torch.randn(batch, N)
x_test /= tla.norm(x_test, dim=0)

for i in range(5_000):
    optimizer.zero_grad()
    
    # Compute smoother
    P = S @ Agg
    
    loss = 0.
    for j in range(batch):
        # Random initial guess and one multigrid iteration
        x0 = x_test[j]
        x1 = mgrid(P, A, x0, b)

        # compute loss as rel residual of x1 / rel residual of x0
        r0 = tla.norm(b-A@x0) / tla.norm(b)
        r1 = tla.norm(b-A@x1) / tla.norm(b)

        loss += (r1 / r0) / batch
    loss.backward()
    
    optimizer.step()
    
    lh.append(loss.item())
    if i % 10 == 0:
        print(i, 'loss:', loss.item())
    if loss.item() < 0.06:
        print(i, 'loss:', loss.item())
        break

In [ ]:
Lh = np.array(lh)

plt.figure()
plt.plot(Lh, 'k')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.grid()

In [ ]:
# Run our candidate smoothers on a random guess

x0 = torch.randn(N); x0 /= tla.norm(x0)

orig_res = None
opt_res = None

with torch.no_grad():
    orig_res = mgrid_solver((sp.eye(N) - 0.4*A) @ Agg, A, x0, b)
    opt_res = mgrid_solver(S @ Agg, A, x0, b)

In [ ]:
plt.title('Comparison of smoothers')
plt.semilogy(orig_res[:5], label='S = A')
plt.semilogy(opt_res[:5], label='Optimized S')
plt.grid()
plt.xlabel('Iteration')
plt.ylabel('$\|b - Ax\| / \|b\|$')
plt.legend()